In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\Machine Learning Implimentation\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\Machine Learning Implimentation\\Chicken-Disease-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import Union

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Union[Path, str]
    source_url: str
    local_data_file: Union[Path, str]
    unzip_dir: Union[Path, str]


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root]
    )
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            fileName, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{fileName} download with following info: \ng{headers}")
        else: 
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extrats the zip file into data directory
        Function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)
        

In [10]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-07 16:44:54,878: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-07 16:44:54,881: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-07 16:44:54,883: INFO: common: created directory at: artifacts]
[2023-07-07 16:44:54,885: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-07 16:44:58,097: INFO: 2007847718: artifacts/data_ingestion/data.zip download with following info: 
gConnection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EE6E:3C22:1223A0:197D04:64A7F3B0
Accept-Ranges: bytes
Date: Fri, 07 Jul 2023 11:14:57 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830041-CCU
X-Cache: M